In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import RepeatedKFold

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

import tensorflow as tf

In [2]:
data = pd.read_csv("Train_Data.csv")
data.head()

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,01-08-2020,campaign 1,adgroup 1,ad 1,24,6,0.08,0,0.00
1,01-08-2020,campaign 1,adgroup 2,ad 1,1,0,0.00,0,0.00
2,01-08-2020,campaign 1,adgroup 3,ad 1,13,4,0.04,0,0.00
3,01-08-2020,campaign 1,adgroup 4,ad 1,5,4,0.08,0,0.00
4,01-08-2020,campaign 1,adgroup 1,ad 2,247,126,1.29,4,925.71


In [3]:
print("shape = ",data.shape, "\n")
# df.info()

shape =  (4571, 9) 



In [4]:
data.describe()

,impressions,clicks,cost,conversions,revenue
count,4571.000000,4571.000000,4571.000000,4571.000000,4571.000000
mean,92.635747,43.340844,2.800834,1.339094,231.865340
std,198.349737,100.164913,14.361618,5.223922,1091.742763
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,2.000000,0.040000,0.000000,0.000000
50%,16.000000,7.000000,0.210000,0.000000,0.000000
75%,84.000000,38.000000,1.350000,1.000000,92.600000
max,3239.000000,1771.000000,531.250000,94.000000,20515.410000


In [5]:
test_df = pd.read_csv('Test_Data.csv')

Xtest2 = test_df.drop(['date','ad'],axis=1)
col = ['campaign','adgroup','impressions','clicks','cost','conversions']  # ,'adnew','day','month'
Xtest2= Xtest2[col]
Xtest2.head(7)

,campaign,adgroup,impressions,clicks,cost,conversions
0,campaign 1,adgroup 1,121,49,0.58,1
1,campaign 1,adgroup 3,22,12,0.17,0
2,campaign 1,adgroup 4,5,3,0.05,0
3,campaign 1,adgroup 2,2,1,0.01,0
4,campaign 1,adgroup 2,3,1,0.01,0
5,campaign 1,adgroup 4,4,2,0.03,0
6,campaign 1,adgroup 1,276,123,1.67,1


In [6]:
model =GradientBoostingRegressor(learning_rate=0.22)


In [7]:
X = data.drop(['date','revenue','ad'],axis=1)
y = data.revenue


sc = StandardScaler()
# X = sc.fit_transform(X)
# Xtest2 = sc.transform(Xtest2)
cols =['impressions','clicks','cost','conversions']
X[cols] = sc.fit_transform(X[cols])

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)
X_train.head()

,campaign,adgroup,impressions,clicks,cost,conversions
416,campaign 1,adgroup 2,-0.436831,-0.402788,-0.187383,-0.256367
1725,campaign 1,adgroup 1,-0.411620,-0.382819,-0.135155,-0.256367
4442,campaign 1,adgroup 3,0.490925,0.465874,-0.114264,0.126529
796,campaign 1,adgroup 4,-0.462041,-0.432742,-0.195043,-0.256367
4082,campaign 1,adgroup 4,-0.431788,-0.412773,-0.192258,-0.256367


In [8]:
Xtest2[cols] =sc.transform(Xtest2[cols])
Xtest2.head()

,campaign,adgroup,impressions,clicks,cost,conversions
0,campaign 1,adgroup 1,0.143017,0.056505,-0.154654,-0.064919
1,campaign 1,adgroup 3,-0.356156,-0.312927,-0.183205,-0.256367
2,campaign 1,adgroup 4,-0.441873,-0.402788,-0.191562,-0.256367
3,campaign 1,adgroup 2,-0.456999,-0.422758,-0.194347,-0.256367
4,campaign 1,adgroup 2,-0.451957,-0.422758,-0.194347,-0.256367


In [9]:
columns_trans1 = make_column_transformer((OrdinalEncoder(),['campaign','adgroup']),remainder='passthrough')

In [10]:
pipe2 = make_pipeline(columns_trans1,model)
pipe2.fit(X_train,y_train)


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['campaign', 'adgroup'])])),
                ('gradientboostingregressor',
                 GradientBoostingRegressor(learning_rate=0.22))])

In [11]:
print("MAE = ",mean_absolute_error(y_test,pipe2.predict(X_test)) )
print('r2 = ', r2_score(y_test,pipe2.predict(X_test)))


MAE =  71.46835651837682
r2 =  0.9086022642582181


In [12]:
res = pd.DataFrame(pipe2.predict(Xtest2),columns=['revenue'])
# res[res['revenue']<0] =0
res.to_csv('resulttemp.csv')
res.head(7)

,revenue
0,154.211914
1,-0.461690
2,-0.461690
3,-0.461690
4,-0.461690
5,-0.461690
6,175.439429
